In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from household_package.data import call_data_url
from household_package.clean import clean_data
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from household_package.model import get_xy
from household_package.clean import filter_data
from household_package.registry import load_model
import pandas as pd
from household_package.registry import save_model
import pickle

In [98]:
BQ_DATASET

In [99]:
BUCKET_NAME="01_household_energy"
LOCAL_REGISTRY_PATH = "../model_h5/loaded/"


In [3]:
df_raw = call_data_url()

In [50]:
user_input = user_input = {
  "TYPEHUQ": 1,
  "NHSLDMEM": 1,
  "state_name": "Alabama",
  "REGIONC": "SOUTH",
  "BA_climate": "Hot-Humid",
  "SQFTEST": 240,
  "STORIES": 1,
  "YEARMADERANGE": 1,
  "NCOMBATH": 0,
  "NHAFBATH": 0,
  "TOTROOMS": 1,
  "WALLTYPE": 1,
  "ROOFTYPE": 1,
  "WINDOWS": 1,
  "SWIMPOOL": 1,
  "SOLAR": 1,
  "SMARTMETER": 1,
  "TELLWORK": 1,
  "DESKTOP": 0,
  "NUMLAPTOP": 0,
  "TVCOLOR": 0,
  "DISHWASH": 1,
  "MICRO": 0,
  "NUMFRIG": 0,
  "CWASHER": 1,
  "DRYER": 1,
  "LGTIN1TO4": 0,
  "LGTIN4TO8": 0,
  "LGTINMORE8": 0,
  "AIRCOND": 1,
  "EQUIPM": 3,
  "HEATHOME": 1,
  "NUMPORTEL": 0
}

In [51]:
def make_X_new(user_input):
    """
    This functions takes a dictionary coming from user inputs
    and makes a one row of X for model prediciton.
    """
    ## get all the passed arguments
    #user_inputs = locals().copy()
    X_new = pd.DataFrame({k:[v] for k,v in user_input.items()})
    return X_new


In [42]:
X_new = make_X_new(user_input)

In [28]:
#X_row_new = pd.DataFrame(pd.Series(user_input)).T

In [43]:
X_row_new = clean_data(X_new)

In [44]:
X_row_new

,SWIMPOOL,NUMPORTEL,SOLAR,SMARTMETER,TYPEHUQ,NHSLDMEM,state_name,REGIONC,BA_climate,SQFTEST,...,MICRO,NUMFRIG,CWASHER,DRYER,AIRCOND,EQUIPM,HEATHOME,TOTAL_BATH,TOTAL_COMP,TOTAL_LIGHT
0,1,0,1,1,Mobile,1,Alabama,SOUTH,Hot-Humid,240,...,0,0,1,1,1,Furnace,1,0,0,0


In [59]:
from google.cloud import storage
client = storage.Client()
blobs = list(client.get_bucket(BUCKET_NAME).list_blobs(prefix="model"))

In [60]:
blobs

[<Blob: 01_household_energy, models/baseline_20231129-122043.pkl, 1701256844126884>,
 <Blob: 01_household_energy, models/baseline_20231129-131320.pkl, 1701260000941981>,
 <Blob: 01_household_energy, models/baseline_20231130-102940.pkl, 1701337090569311>]

In [101]:
latest_blob = max(blobs, key=lambda x: x.updated)
#print(latest_blob.name)
latest_model_path_to_save = os.path.join(LOCAL_REGISTRY_PATH, latest_blob.name)
print(latest_model_path_to_save)
latest_blob.download_to_filename(latest_model_path_to_save)
# for sklearn (baseline)
with open(latest_model_path_to_save , 'rb') as f:
    latest_model = pickle.load(f)

../model_h5/loaded/models/baseline_20231130-102940.pkl


In [70]:
latest_model.predict(X_row_new)

array([11040.])

In [31]:

#percentile_min = df_raw['KWH'].quantile(0.005)
#percentile_max = df_raw['KWH'].quantile(0.995)

# Filter the DataFrame based on the specified percentiles
#filtered_df = df_raw[(df_raw['KWH'] >= percentile_min) & (df_raw['KWH'] <= percentile_max)]
#filtered_df

In [4]:
df = clean_data(df_raw)

In [8]:
df.describe().iloc[1,:]

SWIMPOOL               2
NUMPORTEL             10
SOLAR                  2
SMARTMETER             2
REGIONC                4
state_name            51
BA_climate             8
TYPEHUQ                5
STORIES                4
YEARMADERANGE          9
TOTROOMS              15
WALLTYPE               8
ROOFTYPE               7
WINDOWS                7
NUMFRIG               10
MICRO                  4
DISHWASH               2
CWASHER                2
DRYER                  2
TVCOLOR               14
TELLWORK               2
HEATHOME               2
EQUIPM                 9
AIRCOND                2
NHSLDMEM               7
SQFTEST              469
KWH              18392.0
TOTAL_BATH             6
TOTAL_COMP            16
TOTAL_LIGHT          106
Name: unique, dtype: object

In [12]:
X , y = get_xy(df)

X_train, X_test, y_train, y_test =  train_test_split(X,y, test_size=0.3)

In [6]:
to_ohe_encode = ['REGIONC', 'state_name','BA_climate','TYPEHUQ',
                     'YEARMADERANGE','WALLTYPE','ROOFTYPE','WINDOWS','EQUIPM']

to_scale = ["NUMPORTEL", "STORIES","SQFTEST",
            "TOTROOMS", "NUMFRIG", "MICRO", "TVCOLOR","NHSLDMEM",
            "TOTAL_BATH", "TOTAL_COMP", "TOTAL_LIGHT" ]

min_max = MinMaxScaler()
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')


preprocessor = ColumnTransformer(transformers=[
                                            ('min_max', min_max, to_scale),
                                            ('ohe', ohe, to_ohe_encode)],
                                            remainder = "passthrough")

model = LinearRegression()

pipe = Pipeline([('prep', preprocessor), ('model', model)])

In [23]:
model = pipe.fit(X_train, y_train)

In [46]:
model.score(X_test,y_test)

0.5055383972861784

In [45]:
model.predict(X_row_new)

array([11760.])

In [34]:
X = df.drop('KWH',axis=1)

In [43]:
pd.DataFrame(X.iloc[0,:]).T

,SWIMPOOL,NUMPORTEL,SOLAR,SMARTMETER,REGIONC,state_name,BA_climate,TYPEHUQ,STORIES,YEARMADERANGE,...,TVCOLOR,TELLWORK,HEATHOME,EQUIPM,AIRCOND,NHSLDMEM,SQFTEST,TOTAL_BATH,TOTAL_COMP,TOTAL_LIGHT
0,0,0,0,0,WEST,New Mexico,Mixed-Dry,Single_detached,1,70-79,...,3,0,1,Furnace,1,2,2100,3,1,17


In [44]:
pipe.predict(pd.DataFrame(X.iloc[0,:]).T)

array([10640.])

In [9]:
from sklearn.ensemble import RandomForestRegressor

to_ohe_encode = ['REGIONC', 'state_name','BA_climate','TYPEHUQ',
                     'YEARMADERANGE','WALLTYPE','ROOFTYPE','WINDOWS','EQUIPM']

to_scale = ["NUMPORTEL", "STORIES","SQFTEST",
            "TOTROOMS", "NUMFRIG", "MICRO", "TVCOLOR","NHSLDMEM",
            "TOTAL_BATH", "TOTAL_COMP", "TOTAL_LIGHT" ]

min_max = MinMaxScaler()
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')


preprocessor = ColumnTransformer(transformers=[
                                            ('min_max', min_max, to_scale),
                                            ('ohe', ohe, to_ohe_encode)],
                                            remainder = "passthrough")

model = RandomForestRegressor()

pipe_2 = Pipeline([('prep', preprocessor), ('model', model)])

In [13]:
pipe_2.fit(X_train,y_train)

Pipeline(steps=[('prep',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('min_max', MinMaxScaler(),
                                                  ['NUMPORTEL', 'STORIES',
                                                   'SQFTEST', 'TOTROOMS',
                                                   'NUMFRIG', 'MICRO',
                                                   'TVCOLOR', 'NHSLDMEM',
                                                   'TOTAL_BATH', 'TOTAL_COMP',
                                                   'TOTAL_LIGHT']),
                                                 ('ohe',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['REGIONC', 'state_name',
                                                   'BA_climate', 'TYPEHUQ',
                                                   'YEARMADERANGE', 'WALLTYPE',
                                                   'ROOFTYPE', 'WINDOWS',
                                                   'EQUIPM'])])),
                ('model', RandomForestRegressor())])

In [15]:
pipe_2.score(X_test,y_test)

0.46195432428379035

In [25]:
preprocessor.fit(X_train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('min_max', MinMaxScaler(),
                                 ['NUMPORTEL', 'STORIES', 'SQFTEST', 'TOTROOMS',
                                  'NUMFRIG', 'MICRO', 'TVCOLOR', 'NHSLDMEM',
                                  'TOTAL_BATH', 'TOTAL_COMP', 'TOTAL_LIGHT']),
                                ('ohe',
                                 OneHotEncoder(handle_unknown='ignore',
                                               sparse_output=False),
                                 ['REGIONC', 'state_name', 'BA_climate',
                                  'TYPEHUQ', 'YEARMADERANGE', 'WALLTYPE',
                                  'ROOFTYPE', 'WINDOWS', 'EQUIPM'])])

In [26]:
preprocessor.get_feature_names_out()

array(['min_max__NUMPORTEL', 'min_max__STORIES', 'min_max__SQFTEST',
       'min_max__TOTROOMS', 'min_max__NUMFRIG', 'min_max__MICRO',
       'min_max__TVCOLOR', 'min_max__NHSLDMEM', 'min_max__TOTAL_BATH',
       'min_max__TOTAL_COMP', 'min_max__TOTAL_LIGHT',
       'ohe__REGIONC_MIDWEST', 'ohe__REGIONC_NORTHEAST',
       'ohe__REGIONC_SOUTH', 'ohe__REGIONC_WEST',
       'ohe__state_name_Alabama', 'ohe__state_name_Alaska',
       'ohe__state_name_Arizona', 'ohe__state_name_Arkansas',
       'ohe__state_name_California', 'ohe__state_name_Colorado',
       'ohe__state_name_Connecticut', 'ohe__state_name_Delaware',
       'ohe__state_name_District of Columbia', 'ohe__state_name_Florida',
       'ohe__state_name_Georgia', 'ohe__state_name_Hawaii',
       'ohe__state_name_Idaho', 'ohe__state_name_Illinois',
       'ohe__state_name_Indiana', 'ohe__state_name_Iowa',
       'ohe__state_name_Kansas', 'ohe__state_name_Kentucky',
       'ohe__state_name_Louisiana', 'ohe__state_name_Maine',
      

In [27]:
X_train_prep = preprocessor.transform(X_train)

In [19]:
model = RandomForestRegressor()

In [21]:
model.fit(X_train_prep, y_train)

RandomForestRegressor()

In [23]:
X_train_

array([[0.0, 0.0, 0.06707317073170732, ..., 0, 1, 1],
       [0.0, 0.3333333333333333, 0.05826558265582656, ..., 1, 1, 1],
       [0.0, 0.0, 0.08536585365853658, ..., 0, 1, 1],
       ...,
       [0.0, 0.0, 0.06504065040650406, ..., 0, 1, 1],
       [0.2222222222222222, 0.0, 0.0921409214092141, ..., 0, 1, 1],
       [0.0, 0.6666666666666667, 0.10433604336043362, ..., 0, 1, 1]],
      dtype=object)

In [43]:
pd.DataFrame(model.feature_importances_, index=preprocessor.get_feature_names_out()).sort_values(by=0, ascending=False).head(10)

,0
min_max__SQFTEST,0.222017
min_max__NHSLDMEM,0.057220
ohe__REGIONC_SOUTH,0.056915
min_max__TVCOLOR,0.050906
remainder__SWIMPOOL,0.050628
min_max__TOTAL_LIGHT,0.047382
min_max__TOTROOMS,0.043606
min_max__NUMFRIG,0.040830
ohe__EQUIPM_Central_heat,0.036158
min_max__TOTAL_COMP,0.029290
